# 1. Merge all files

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from xgboost import XGBRegressor
from xgboost import XGBClassifier
import pandas as pd

import selenium
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
from datetime import datetime, timedelta
import os

import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [27]:
folder_path = "D:\OneDrive\KiotViet\Python_for_work\KFinance\CSV_crawl_data"
file_list = os.listdir(folder_path)

merged_df = pd.DataFrame()

# Loop qua từng file CSV và merge vào DataFrame
for file in file_list:
    if "cho-thue-nha-mat-pho_order" in file or "cho-thue-shophouse-nha-pho-thuong-mai_order" in file or "cho-thue-kho-nha-xuong-dat_order" in file:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        merged_df = pd.concat([merged_df, df], ignore_index=True)

print(len(merged_df))

8376


In [28]:
merged_df = merged_df.dropna()
merged_df.to_csv(r"D:\OneDrive\KiotViet\Python_for_work\KFinance\CSV_crawl_data\all_bds_bdscomvn.csv", index=False, mode = 'w', encoding="utf-8-sig")

# 2. Clean data 

In [29]:
df = pd.read_csv(r"D:\OneDrive\KiotViet\Python_for_work\KFinance\CSV_crawl_data\all_bds_bdscomvn.csv")
df.head()
print(f"The total rows of dataframe is {len(df)}")

The total rows of dataframe is 8368


2.0 Xử lý diện tích

In [30]:
# area_pattern = re.compile(r'(\d+) m²')
# df['area_value'] = df['Area'].apply(lambda x: int(area_pattern.search(x).group(1)) if area_pattern.search(x) else None)

df['area_value'] = df['Area'].apply(lambda x: float(x.replace(',', '.').replace(' m²', '').replace('.', '')))
df['area_value']


0        40610.0
1       104700.0
2        15000.0
3        50000.0
4          150.0
          ...   
8363       160.0
8364       120.0
8365      1400.0
8366       116.0
8367       120.0
Name: area_value, Length: 8368, dtype: float64

2.1 Xử lý price

In [31]:
# Tạo cột 'price_per_m2'

# 1. Price ghi: 70 nghìn / m2 => Cột 'price_per_m2' thành 70000
pattern = re.compile(r'(\d+) nghìn/m²')
df['price_per_m2'] = df['Price'].apply(lambda x: int(pattern.search(x).group(1)) if pattern.search(x) else None)*1000

#2.Nếu Price ghi X triệu/tháng và Area ghi Y m2 thì => Cột 'price_per_m2' thành x / Y
def convert_price_string_to_float(price_string):
    return float(price_string.replace(',', '.'))

price_pattern = re.compile(r'([\d,]+) triệu/tháng')

df['price_value_trieu'] = df['Price'].apply(lambda x: convert_price_string_to_float(price_pattern.search(x).group(1)) if price_pattern.search(x) else None)
df['price_per_m2_trieu'] = 1000000 * df['price_value_trieu'] / df['area_value']

#3.Nếu Price ghi X tỷ/tháng và Area ghi Y m2 thì => Cột 'price_per_m2' thành x / Y

price_pattern_bil = re.compile(r'([\d,]+) tỷ')

df['price_value_ty'] = df['Price'].apply(lambda x: convert_price_string_to_float(price_pattern_bil.search(x).group(1)) if price_pattern_bil.search(x) else None)
df['price_per_m2_ty'] = 1000000000 * df['price_value_ty'] / df['area_value']

#3. Nếu Price là thoả thuận thì bỏ qua khỏi df luôn
df = df[df['Price'] != 'Giá thỏa thuận']



In [32]:
#4. Từ các cột "price_per_m2", "price_per_m2_trieu", "price_per_m2_ty" ta tìm được price_per_m2_final

def calculate_price_per_m2_final(row):
    if not pd.isna(row['price_per_m2']):
        return row['price_per_m2']
    elif not pd.isna(row['price_per_m2_trieu']):
        return row['price_per_m2_trieu']
    elif not pd.isna(row['price_per_m2_ty']):
        return row['price_per_m2_ty']
    else:
        return np.nan

# Áp dụng hàm tùy chỉnh để tạo cột mới
df['price_per_m2_final'] = df.apply(calculate_price_per_m2_final, axis=1)

In [33]:
#5. Xử lý thành phố
df[['district', 'city']] = df['Location'].str.split(', ', expand=True)

In [34]:
df['address'] = df['Title']
df.head()

,Title,Price,Area,Location,type,area_value,price_per_m2,price_value_trieu,price_per_m2_trieu,price_value_ty,price_per_m2_ty,price_per_m2_final,district,city,address
2,Chính chủ cho thuê nhà xưởng có diện tích từ 3...,52 nghìn/m²,15.000 m²,"Hưng Yên, Hưng Yên",cho-thue-kho-nha-xuong-dat,15000.0,52000.0,NaN,NaN,NaN,NaN,52000.0,Hưng Yên,Hưng Yên,Chính chủ cho thuê nhà xưởng có diện tích từ 3...
5,Cho thuê kho có sẵn PCCC tự động tại KCN Vĩnh ...,100 nghìn/m²,250 m²,"Bình Chánh, Hồ Chí Minh",cho-thue-kho-nha-xuong-dat,250.0,100000.0,NaN,NaN,NaN,NaN,100000.0,Bình Chánh,Hồ Chí Minh,Cho thuê kho có sẵn PCCC tự động tại KCN Vĩnh ...
8,"Kho riêng (tự quản), kho chung có thủ kho quản...",50 nghìn/m²,1.000 m²,"Quận 7, Hồ Chí Minh",cho-thue-kho-nha-xuong-dat,1000.0,50000.0,NaN,NaN,NaN,NaN,50000.0,Quận 7,Hồ Chí Minh,"Kho riêng (tự quản), kho chung có thủ kho quản..."
10,"Chính chủ cho thuê kho 2000m2, đường xe công 2...",170 triệu/tháng,2.000 m²,"Bình Chánh, Hồ Chí Minh",cho-thue-kho-nha-xuong-dat,2000.0,NaN,170.0,85000.0,NaN,NaN,85000.0,Bình Chánh,Hồ Chí Minh,"Chính chủ cho thuê kho 2000m2, đường xe công 2..."
13,Cho thuê nhà xưởng xây dựng theo yêu cầu đảm b...,80 nghìn/m²,5.000 m²,"Đông Anh, Hà Nội",cho-thue-kho-nha-xuong-dat,5000.0,80000.0,NaN,NaN,NaN,NaN,80000.0,Đông Anh,Hà Nội,Cho thuê nhà xưởng xây dựng theo yêu cầu đảm b...


In [35]:
list_api_keys = ["1EotnhALO_Xf7d5mlOVefECdfpbgPG_Dik0TeOcQauQ",
"92hxCAU523jWUFRXMhM-p3w66P3PxYfY_tZm9pPeShU",
"KwvOafjkobVhF8Iz63lMyNooS-hjaYgmHdZrqVVVHWw",
"N5qb2q5dSfly3fnqWMsDPqVMWUd1yRbTBxbT34hvw_g",
"SsQNhtNfemZJr_FzCYZlub29VeTTjymT3Qm6syC50W8",
"br4m9W3KMhrPa51Vru5M1mWJYCfaVFvWiP3kB2gEQUA",
"spdVYGb-b6cgokZBOuwGG4b3WbmgsxQqv7tudjdOfLc",
"0zeR-GEwo5Od-poIZYTGrw-I80fC2C0b0q4F1Boa8oc",
"UfHD68dH4O9ld-NS22Se6Bls98P9aPZhFq4hA_H7-74",
"c6NrKs_BpAX4OdRFKAVSDeUaQTjSpNF8dCxjcLT6AfE",
"WCpDyLiAACoL9zwSFQQa_HboP-HdWXSM601W1N02aeg",
"faJrm5egL12NmLivrDsbqz44I47DhbJQUIg89L3TdIo",
"cdXQHZ3zzsZv-n04kjimcml7e3BRKcR2H3wl8XPi6SU",
"RBRLSkMgpwy6QF0jKsXs-jXEliqtAwwD_jd0DbHrEN8",
"Dd-n66bTKOIKeQ0H6nRvIOMWatnJ_viPxKMNGpxsFa4",
"q-Ao_RRfZNyfQPp2XsnYGbrOItF6SlXN0jObPIKVnk",
"ufLjrWIrvjyLHzL1TZIEOZjKpgolDMBUPXUwoIblgs0",
"tvjh1g0Xygp6DMsH8n5hX41jgbNh3VE61MsnWHYkfZw",
"d_8_FhktHjeMh91gB9bBKGHgMJFC6auZqHThhlHL9Hw",
"P1RAwTjkXvZUQLjHyi06Zm1YP1KeaZMJdjaNN1ZeOeE",
"DQV3KQ2U9DLoro0jMQUoIr-PQMKBpKIEpOqXzF1A498",
"6IcCytBh4PkKOC3YE2Smw4IGdAyTD7XTtsmgtxF-kdY",
"PgxoArILmRDFQreVp3pC6bexYfcKIVwMMN4oKpa8VOo",
"mwA5L1GvWFvnwIYL-U1kP0RDsZ61ElOKAH7wnbsTs4M"]

In [36]:
# Hàm gọi api here để lấy thông tin vị trí từ địa chỉ chi tiết
import requests

def get_location(address, api_key ):
    #api_key = "mwA5L1GvWFvnwIYL-U1kP0RDsZ61ElOKAH7wnbsTs4M"

    url = f'https://geocode.search.hereapi.com/v1/geocode?q={address}&apiKey={api_key}'
    
    try:
        # Gửi yêu cầu API
        response = requests.get(url)
        
        # Nếu API key đã hết hạn, thử API key tiếp theo trong danh sách

        if response.status_code == 429:
            return {"lat": "", "lng": "", "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "False" }
        else: 
            data = response.json()    
            # Xử lý kết quả và trả về
            if data == {'items': []}: 
                list_address = {"lat": "", "lng": "", "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "True" }
            else: 
                result = data['items'][0]
                position = result['position']
                lat = position.get('lat', '')
                long = position.get('lng', '')
                address_info = result['address']
                city = address_info.get('county', '')
                district = address_info.get('city', '')
                ward = address_info.get('district', '')
                street = address_info.get('street', '')
                houseNumber = address_info.get('houseNumber', '')
                list_address = {"lat": lat, "lng": long, "House_number": houseNumber, "Street": street, "Ward": ward, "District": district, "City": city, "progress": "True" }
    except:
           # Nếu thử xảy ra lỗi trả ra tập rỗng và progess là False đánh dấu những case chưa xử lí
        list_address = {"lat": "", "lng": "", "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "False" }
           
    return list_address

# Hàm xử lí dữ liệu đầu vào là dataframe
def out_put(df, api_key):
    house_numbers = []
    streets = []
    wards = []
    districts = []
    cities = []
    progresses = []
    
    for address in df["address"]:
        address_info = get_location(address,api_key)
        house_numbers.append(address_info["House_number"])
        streets.append(address_info["Street"])
        wards.append(address_info["Ward"])
        districts.append(address_info["District"])
        cities.append(address_info["City"])
        progresses.append(address_info["progress"])
    
    df["House_number"] = house_numbers
    df["Street"] = streets
    df["Ward"] = wards
    df["District"] = districts
    df["City"] = cities
    df["progress"] = progresses

    return df

In [37]:
df_test = df

In [38]:
# Calculate the number of groups
num_groups = len(df_test) // 1000

# Calculate the number of remaining rows
remaining_rows = len(df_test) % 1000

# Create a list to store the new dataframes
dfs = []

# Split the dataframe into groups of two rows each
for i in range(num_groups):
    start_index = i * 1000
    end_index = start_index + 1000
    df_group = df_test[start_index:end_index].copy()
    dfs.append(df_group)

# Add the remaining rows to the last dataframe
if remaining_rows > 0:
    last_df = df_test[-remaining_rows:].copy()
    dfs.append(last_df)

In [39]:
# tiến hành chạy multi thread
import threading
import time

final = []
def out_put_with_time(df, api_key):
    start_time = time.time()
    out_put(df, api_key)
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Time: {execution_time/60} phút")

for df , api_key in zip(dfs, list_api_keys):
    thread = threading.Thread(target=out_put_with_time, args=(df,api_key))
    thread.start()
    thread.join()
    final.append(df)

print("Done")

Time: 6.526620554924011 phút
Time: 6.256240733464559 phút
Time: 6.41014103492101 phút
Time: 6.675470264752706 phút
Time: 6.685437222321828 phút
Time: 6.55020964940389 phút
Time: 6.54061461687088 phút
Time: 3.109566831588745 phút
Done


In [46]:
merged_df = pd.concat(final, ignore_index=True)

#check case false
# merged_df[merged_df["progress"] == "False"]
merged_df.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7472 entries, 0 to 7471
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Title               7472 non-null   object 
 1   Price               7472 non-null   object 
 2   Area                7472 non-null   object 
 3   Location            7472 non-null   object 
 4   type                7472 non-null   object 
 5   area_value          7472 non-null   float64
 6   price_per_m2        1026 non-null   float64
 7   price_value_trieu   6413 non-null   float64
 8   price_per_m2_trieu  6413 non-null   float64
 9   price_value_ty      30 non-null     float64
 10  price_per_m2_ty     30 non-null     float64
 11  price_per_m2_final  7469 non-null   float64
 12  district            7472 non-null   object 
 13  city                7472 non-null   object 
 14  address             7472 non-null   object 
 15  House_number        7472 non-null   object 
 16  Street

In [52]:
merged_df['City'] = np.where(merged_df['City'].str.contains('[a-zA-Z]'),merged_df['City'],merged_df['city'])
merged_df['District'] = np.where(merged_df['District'].str.contains('[a-zA-Z]'),merged_df['District'],merged_df['district'])
merged_df['price_value_trieu'] = merged_df['price_per_m2_final']*merged_df['area_value']/1000000


In [2]:
the_end = merged_df.filter(['City','District','Ward','Street','address','area_value','price_value_trieu','price_per_m2_final'],axis = 1)
the_end = the_end.dropna()

NameError: name 'merged_df' is not defined

In [56]:
the_end.to_csv(r"D:\OneDrive\KiotViet\Python_for_work\KFinance\CSV_crawl_data\final_bdscomvn.csv", index=False, mode = 'w', encoding="utf-8-sig")